In [28]:
import numpy as np

def matriz(term, ref):
    """
    Metodo de las transparencias
    """
    vref = np.array(list(ref))
    return np.vstack([vref != letter for letter in term]) + 0

def dp_levenshtein_backwards_threshold(term, ref, threshold):
    """
    Calcula la distancia de Levenshtein entre las cadenas term y ref
    con un umbral maximo threshold
    """
    mat = matriz(term, ref)
    res = np.empty((len(term)+1,len(ref)+1))
    res[:] = np.NaN

    # Lista de elementos que nos dice True si hay que seguir calculando los valores de una columna o False si ya sabemos
    # que superarán el threshold.
    j_valido = [True] * (len(ref)+1)
    
    for i in range(0,len(term)+1):
        for j in range(0,len(ref)+1):

            # Hace la comprobación de si el valor de la columna se debe calcular o se sabe ya que va a superar el threshold
            if not j_valido[j]:
                continue

            if i==0 or j==0:
                aux_res = res[i,j] if not np.isnan(res[i,j]) else 0 
                res[i,j] = aux_res + i + j if aux_res + i + j < threshold else None

            else:
                aux_list = [
                    mat[i-1,j-1] + res[i-1,j-1] if mat[i-1,j-1] + res[i-1,j-1] < threshold else None,
                    1 + res[i-1,j] if 1 + res[i-1,j] < threshold else None,
                    1 + res[i,j-1] if 1 + res[i,j-1] < threshold else None]
                
                # Encuentra el mínimo en una lista con valores que son None
                res[i,j] = min(filter(lambda x: x is not None, aux_list)) if any(aux_list) else None
            
            # Si el elemnto es nan y los anteriores elementos de la fila son aceptados hacemos un break para 
            # no realizar iteraciones extra que sabemos que excederan el threshold slatando a la siguiente fila.
            if np.isnan(res[i, j]) and np.any(~np.isnan(res)[i]):
                break

            # Si el elemnto es nan y los anteriores elementos de la columna son aceptado modificamos el valor en j_valido
            # para así saber que cuando se llegue a una columna con valor j no hay que calcularlo ya que superar el threshold
            if np.isnan(res[i, j]) and np.any(~np.isnan(res)[:,j]):
                j_valido[j] = False
            
            # Prints para debugging
            print("i:", i,"j:", j, "||", "valor:", res[i,j])
            print(res)
    return res[len(term),len(ref)]


def dp_restricted_damerau_backwards_threshold(term, ref, threshold):
    """
    Calcula la distancia de Damerau Levenshtein Restringida entre las cadenas x y y
    """
    # Simula el infinito
    INF = len(term) + len(ref)

    mat = matriz(term, ref)
    res = np.empty((len(term)+1,len(ref)+1))
    res[:] = np.NaN
    # res = np.zeros(shape=(len(term)+1,len(ref)+1))

    # Lista de elementos que nos dice True si hay que seguir calculando los valores de una columna o False si ya sabemos
    # que superarán el threshold.
    j_valido = [True] * (len(ref)+1)

    for i in range(0,len(term)+1):
        for j in range(0,len(ref)+1):

            # Hace la comprobación de si el valor de la columna se debe calcular o se sabe ya que va a superar el threshold
            if not j_valido[j]:
                continue

            if i==0 or j==0:
                aux_res = res[i,j] if not np.isnan(res[i,j]) else 0 
                res[i,j] = aux_res + i + j if aux_res + i + j < threshold else None
            elif i == 1 or j == 1:
                aux_list = [
                    mat[i-1,j-1] + res[i-1,j-1] if mat[i-1,j-1] + res[i-1,j-1] < threshold else None,
                    1 + res[i-1,j] if 1 + res[i-1,j] < threshold else None,
                    1 + res[i,j-1] if 1 + res[i,j-1] < threshold else None]
                
                # Encuentra el mínimo en una lista con valores que son None
                res[i,j] = min(filter(lambda x: x is not None, aux_list)) if any(aux_list) else None
            else:
                aux_list = [
                    mat[i-1,j-1] + res[i-1,j-1] if mat[i-1,j-1] + res[i-1,j-1] < threshold else None,
                    1 + res[i-1,j] if 1 + res[i-1,j]  < threshold else None,
                    1 + res[i,j-1] if 1 + res[i,j-1]  < threshold else None,
                    1 + res[i-2,j-2] + (mat[i-2,j-1] + mat[i-1,j-2]) * INF if 1 + res[i-2,j-2] + (mat[i-2,j-1] + mat[i-1,j-2]) * INF < threshold else None]
                
                # Encuentra el mínimo en una lista con valores que son None
                res[i,j] = min(filter(lambda x: x is not None, aux_list)) if any(aux_list) else None
            print("i:", i,"j:", j, "||", "valor:", res[i,j])
            print(res)
    return res[len(term),len(ref)]

In [34]:
for term,ref in [("ccamal", "cacmapasdl")]:
    print(term,ref,dp_restricted_damerau_backwards_threshold(term,ref,6))

 nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]]
i: 1 j: 10 || valor: nan
[[ 0.  1.  2.  3.  4.  5. nan nan nan nan nan]
 [ 1.  0.  1.  2.  3.  4.  5. nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]]
i: 2 j: 0 || valor: 2.0
[[ 0.  1.  2.  3.  4.  5. nan nan nan nan nan]
 [ 1.  0.  1.  2.  3.  4.  5. nan nan nan nan]
 [ 2. nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan]]
i: 2 j: 1 || valor: 1.0
[[ 0.  1.  2.  3.  4.  5. nan nan nan nan nan]
 [ 1.  0.  1.  2.  3.  4.  5. nan nan nan nan]
 [ 2.  1. nan nan nan nan

In [79]:
x="hola"
y="chola"
M = np.zeros((len(x) + 1, len(y) + 1))

# COMENTARIO DE DAVID: 
for i in range(1, len(x) + 1):
    M[i, 0] = i
for j in range(1, len(y) + 1):
    M[0, j] = j
M

array([[0., 1., 2., 3., 4., 5.],
       [1., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0.]])

In [84]:
M = np.zeros((len(x) + 1, len(y) + 1))
M[0] = np.arange(len(y)+1)
M[:][:,0] = np.arange(len(x)+1)
M
matriz("hola", "hoilas")

array([[0, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 1]])

In [1]:
from tarea2.distances_with_threshold import dp_intermediate_damerau_backwards_with_threshold

In [5]:
dp_intermediate_damerau_backwards_with_threshold('hola','holo',2)

[[ 0.  1.  2.  3.  4.]
 [ 1. inf inf inf inf]
 [ 2. inf inf inf inf]
 [ 3. inf inf inf inf]
 [ 4. inf inf inf inf]]
[[ 0.  1.  2.  3.  4.]
 [ 1.  0.  1. inf inf]
 [ 2. inf inf inf inf]
 [ 3. inf inf inf inf]
 [ 4. inf inf inf inf]]
[[ 0.  1.  2.  3.  4.]
 [ 1.  0.  1. inf inf]
 [ 2.  1.  0.  1. inf]
 [ 3. inf inf inf inf]
 [ 4. inf inf inf inf]]
[[ 0.  1.  2.  3.  4.]
 [ 1.  0.  1. inf inf]
 [ 2.  1.  0.  1. inf]
 [ 3.  2.  1.  0.  1.]
 [ 4. inf inf inf inf]]
[[ 0.  1.  2.  3.  4.]
 [ 1.  0.  1. inf inf]
 [ 2.  1.  0.  1. inf]
 [ 3.  2.  1.  0.  1.]
 [ 4. inf  2.  1.  1.]]


1.0

In [1]:
from tarea2.distances_with_threshold import dp_levenshtein_backwards_threshold
dp_levenshtein_backwards_threshold('aaaa', 'bbbbb', 3)

[[ 0.  1.  2.  3.  4.  5.]
 [ 1. inf inf inf inf inf]
 [ 2. inf inf inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1. inf inf inf inf]
 [ 2. inf inf inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1.  2. inf inf inf]
 [ 2. inf inf inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1.  2.  3. inf inf]
 [ 2. inf inf inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1.  2.  3. inf inf]
 [ 2. inf inf inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1.  2.  3. inf inf]
 [ 2.  2. inf inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1.  2.  3. inf inf]
 [ 2.  2.  2. inf inf inf]
 [ 3. inf inf inf inf inf]
 [ 4. inf inf inf inf inf]]
[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  1.  2.  3. inf